#Dynamic Topic Modeling with Trump's Tweets

In this notebook we will be using Dynamic Topic Modeling with BERTopic to visualize how topics in Trump's Tweets have evolved over time. These topics will be visualized and thoroughly explored. 

## Dynamic Topic Models
Dynamic topic models can be used to analyze the evolution of topics of a collection of documents over time. 


# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing BERTopic

We start by installing BERTopic from PyPi:

In [1]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# **Data**
For this tutorial, we will be needing to extract all Trump's Tweet from his @realDonalTrump account. We will be removing all retweet and focus on his original tweets. 

Moreover, since we are looking at his tweets over time, we will be saving all timestamps related to his tweets.

In [2]:
import re
import pandas as pd
from datetime import datetime

# Load data
trump = pd.read_csv('/content/tweets_01-08-2021.csv')
print(trump.head())
# Filter
trump.text = trump.apply(lambda row: re.sub(r"http\S+", "", row.text).lower(), 1)
trump.text = trump.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.text.split())), 1)
trump.text = trump.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.text).split()), 1)
trump = trump.loc[(trump.isRetweet == "f") & (trump.text != ""), :]
timestamps = trump.date.to_list()
tweets = trump.text.to_list()

                    id                                               text  \
0    98454970654916608  Republicans and Democrats have both created ou...   
1  1234653427789070336  I was thrilled to be back in the Great city of...   
2  1218010753434820614  RT @CBS_Herridge: READ: Letter to surveillance...   
3  1304875170860015617  The Unsolicited Mail In Ballot Scam is a major...   
4  1218159531554897920  RT @MZHemingway: Very friendly telling of even...   

  isRetweet isDeleted              device  favorites  retweets  \
0         f         f           TweetDeck         49       255   
1         f         f  Twitter for iPhone      73748     17404   
2         t         f  Twitter for iPhone          0      7396   
3         f         f  Twitter for iPhone      80527     23502   
4         t         f  Twitter for iPhone          0      9081   

                  date isFlagged  
0  2011-08-02 18:07:48         f  
1  2020-03-03 01:34:50         f  
2  2020-01-17 03:22:47         f  


In [3]:
tweets[0]

'republicans and democrats have both created our economic problems'

In [11]:
timestamps[:10]

['2011-08-02 18:07:48',
 '2020-03-03 01:34:50',
 '2020-09-12 20:10:58',
 '2020-02-01 16:14:02',
 '2020-01-09 12:24:31',
 '2020-10-23 04:09:59',
 '2020-11-09 19:36:26',
 '2020-10-12 22:22:39',
 '2020-10-23 16:55:29',
 '2020-10-23 15:04:10']

# **Dynamic Topic Modeling**


## Basic Topic Model
To perform Dynamic Topic Modeling with BERTopic we will first need to create a basic topic model using all tweets. The temporal aspect will be ignored as we are, for now, only interested in the topics that reside in those tweets. 

In [4]:
from bertopic import BERTopic
topic_model = BERTopic(min_topic_size=35, verbose=True)
topics, _ = topic_model.fit_transform(tweets)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/1418 [00:00<?, ?it/s]

2022-12-30 17:09:44,741 - BERTopic - Transformed documents to Embeddings
2022-12-30 17:11:00,094 - BERTopic - Reduced dimensionality
2022-12-30 17:11:05,048 - BERTopic - Clustered reduced embeddings


We can then extract most frequent topics:

In [6]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name
0,-1,14905,-1_the_to_of_is
1,0,3086,0_run_president_trump_donald
2,1,1796,1_crowd_carolina_join_thank
3,2,1072,2_border_wall_mexico_immigration
4,3,930,3_entrepreneurs_yourself_think_your
5,4,861,4_hotel_tower_chicago_building
6,5,785,5_china_trade_tariffs_chinese
7,6,630,6_golf_course_scotland_turnberry
8,7,611,7_obamacare_healthcare_repeal_website
9,8,586,8_amp_you_it_to


In [19]:
freq.tail(10)

,Topic,Count,Name
154,153,39,153_venezuela_cuba_maduro_cuban
155,154,38,154_cast_star_season_premieres
156,155,38,155_polls_fake_poll_suppression
157,156,38,156_charity_offer_obama_barack
158,157,37,157_cpac_via_speak_iowa
159,158,36,158_thanks_will_do_ellie
160,159,36,159_lago_mar_palm_beach
161,160,36,160_russia_russian_meddling_didn
162,161,36,161_jobs_promises_kept_jobsnotmobs
163,162,35,162_vote_register_mail_early


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [7]:
topic_model.get_topic(4)

[('hotel', 0.04939930311365253),
 ('tower', 0.038447679729865965),
 ('chicago', 0.036812819028457795),
 ('building', 0.021332623612627777),
 ('sign', 0.020542027219923806),
 ('trump', 0.016912233043930008),
 ('nyc', 0.016479441545443833),
 ('luxury', 0.016362754953488122),
 ('vegas', 0.014443801151414883),
 ('city', 0.014298610317590912)]

In [8]:
topic_model.get_topic(5)

[('china', 0.056036687859996576),
 ('trade', 0.024734103553218784),
 ('tariffs', 0.024265675641908176),
 ('chinese', 0.014270897508020568),
 ('us', 0.011767004828738235),
 ('farmers', 0.011623522502228832),
 ('deal', 0.010679164483821757),
 ('our', 0.010390610131771578),
 ('currency', 0.0096242466453308),
 ('we', 0.009243735066849218)]

We can visualize the basic topics that were created with the Intertopic Distance Map. This allows us to judge visually whether the basic topics are sufficient before proceeding to creating the topics over time. 

In [9]:
fig = topic_model.visualize_topics(); fig

## Topics over Time
Before we start with the Dynamic Topic Modeling step, it is important that you are satisfied with the topics that were created previously. We are going to be using those specific topics as a base for Dynamic Topic Modeling. 

Thus, this step will essentially show you how the topics that were defined previously have evolved over time. 

There are a few important parameters that you should take note of, namely:

* `docs`
  * These are the tweets that we are using
* `timestamps`
  * The timestamp of each tweet/document
* `global_tuning`
  * Whether to average the topic representation of a topic at time *t* with its global topic representation
* `evolution_tuning`
  * Whether to average the topic representation of a topic at time *t* with the topic representation of that topic at time *t-1*
* `nr_bins`
  * The number of bins to put our timestamps into. It is computationally inefficient to extract the topics at thousands of different timestamps. Therefore, it is advised to keep this value below 20. 


In [14]:
topics_over_time = topic_model.topics_over_time(docs=tweets, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

20it [00:06,  3.19it/s]


## Visualize Topics over Time
After having created our `topics_over_time`, we will have to visualize those topics as accessing them becomes a bit more difficult with the added temporal dimension. 

To do so, we are going to visualize the distribution of topics over time based on their frequency. Doing so allows us to see how the topics have evolved over time. Make sure to hover over any point to see how the topic representation at time *t* differs from the global topic representation. 


In [18]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, width=1200, height=700)